In [11]:
import pandas as pd
import numpy as np

def classify_food_group(carbs, protein, fat, fiber):
    kcal = 4*carbs + 4*protein + 9*fat
    if kcal == 0:
        return "Unknown"
    cho_ratio = 4 * carbs / kcal
    protein_ratio = 4 * protein / kcal
    fat_ratio = 9 * fat / kcal

    if cho_ratio > 0.65 and protein_ratio < 0.1:
        return "과일군"
    elif cho_ratio > 0.55 and protein_ratio < 0.25:
        return "곡류군"
    elif protein_ratio > 0.4 and cho_ratio < 0.3:
        return "어육류군"
    elif fat_ratio > 0.8:
        return "지방군"
    elif 0.2 < cho_ratio < 0.5 and 0.2 < protein_ratio < 0.5 and 0.2 < fat_ratio < 0.5:
        return "우유군"
    elif fiber > 2 and kcal < 100:
        return "채소군"
    else:
        return "혼합식품"
    
# 각 항목 점수화 함수 정의
def score_meal(row):

    # 이상적 기준값 설정
    ideal_g_max = 120
    ideal_delta_g = 25
    ideal_gl = 60
    ideal_fii = 0.15
    ideal_fiber = 10
    ideal_ratios = {'CHO': 0.5, 'Protein': 0.2, 'Fat': 0.3}

    # ΔG (낮을수록 좋음, 0~40 사이 정규화)
    delta_g_score = np.clip(row["delta_g"] / 40, 0, 1)

    # g_max (140 이상이면 감점)
    g_max_score = np.clip((row["g_max"] - ideal_g_max) / 40, 0, 1)

    # GL (80 이상이면 감점)
    gl_score = np.clip((row["GL"] - ideal_gl) / 40, 0, 1)

    # Fiber (많을수록 좋음, 10g 이상이면 만점)
    fiber_score = np.clip(row["fiber"] / ideal_fiber, 0, 1)

    # 영양 비율 편차 (Euclidean distance)
    dist_ratio = np.sqrt(
        (row["CHO_ratio"] - ideal_ratios["CHO"])**2 +
        (row["Protein_ratio"] - ideal_ratios["Protein"])**2 +
        (row["Fat_ratio"] - ideal_ratios["Fat"])**2
    )
    ratio_score = np.clip(dist_ratio / 0.5, 0, 1)  # max 0.5 정도로 스케일링

    # 최종 점수 (100점 만점, 감점 구조)
    score = 100 - (
        20 * delta_g_score +
        20 * g_max_score +
        20 * gl_score +
        20 * ratio_score +
        20 * fiber_score
    )
    return round(score, 1)



In [12]:
# 업로드된 파일 경로
file_path = "total_metrics.csv"
df = pd.read_csv(file_path)

# 식품군 분류 적용
df["식품군분류"] = df.apply(
    lambda row: classify_food_group(row["carbs"], row["protein"], row["fat"], row["fiber"]),
    axis=1
)

# 필요한 파생 변수 계산
df["GL"] = df["carbs"] - df["fiber"]
kcal = 4 * df["carbs"] + 4 * df["protein"] + 9 * df["fat"]
df["CHO_ratio"] = 4 * df["carbs"] / kcal
df["Protein_ratio"] = 4 * df["protein"] / kcal
df["Fat_ratio"] = 9 * df["fat"] / kcal

# 좋은 식사 판단 함수
def is_good_meal(row):
    try:
        return (
            row["delta_g"] < 30 and
            row["g_max"] < 140 and
            row["GL"] < 80 and
            row["FII_v1"] < 0.2 and
            0.4 < row["CHO_ratio"] < 0.6 and
            0.15 < row["Protein_ratio"] < 0.3 and
            0.2 < row["Fat_ratio"] < 0.35 and
            row["fiber"] >= 5
        )
    except:
        return False

# 라벨 적용
df["good_meal_label"] = df.apply(is_good_meal, axis=1)
# 점수 계산 후 추가
df["meal_score"] = df.apply(score_meal, axis=1)

In [13]:
# 저장
output_columns = [
    "patient_id", "meal_time", "meal_type", "carbs", "protein", "fat", "fiber",
    "delta_g", "g_max", "GL",
    "CHO_ratio", "Protein_ratio", "Fat_ratio",
    "식품군분류", "good_meal_label", "meal_score"
]

output_path = "evaluated_meals.csv"
df[output_columns].to_csv(output_path, index=False)
output_path

'evaluated_meals.csv'